# Modelling CNN

## Import Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Conv1D, MaxPool1D, Dropout
# from tqdm import tqdm

import numpy as np
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score

import warnings
warnings.filterwarnings('ignore')
import random

np.random.seed(0)
tf.random.set_seed(0)
random.seed(0)

## Function for Average

In [2]:
def Average(lst):
    return sum(lst) / len(lst)

In [3]:
!pip install gdown

In [4]:
!gdown --id 1s-4XNKLxOiHe_uORpbhwF7pVZjCXME0e


/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1s-4XNKLxOiHe_uORpbhwF7pVZjCXME0e
To: /kaggle/working/dataset_50k.csv
100%|███████████████████████████████████████| 6.77M/6.77M [00:00<00:00, 131MB/s]


In [5]:
!gdown https://drive.google.com/uc?id=1fy2o5J5wFEd-ksO6fG9_XVQFssiS-nC4

Downloading...
From: https://drive.google.com/uc?id=1fy2o5J5wFEd-ksO6fG9_XVQFssiS-nC4
To: /kaggle/working/corpus_indonews_twitter.fasttext
100%|███████████████████████████████████████| 12.7M/12.7M [00:00<00:00, 253MB/s]


In [6]:
!gdown https://drive.google.com/uc?id=1pRsOh_uJgRh0aUKd3Et23oLM9CJJNYcl


Downloading...
From (original): https://drive.google.com/uc?id=1pRsOh_uJgRh0aUKd3Et23oLM9CJJNYcl
From (redirected): https://drive.google.com/uc?id=1pRsOh_uJgRh0aUKd3Et23oLM9CJJNYcl&confirm=t&uuid=b8bb6d12-38ec-4e82-a91d-2cc17f249d6d
To: /kaggle/working/corpus_indonews_twitter.fasttext.wv.vectors_ngrams.npy
100%|█████████████████████████████████████████| 800M/800M [00:04<00:00, 180MB/s]


## Import Dataset

In [7]:
df = pd.read_csv('dataset_50k.csv', usecols = ['detokenize', 'Label'])


In [8]:
df = df.dropna()
df

Label                                         detokenize
0          1  aku luka diri aku jalan hidup sulit orang tua ...
1          0   buang pikir negatif laku gelisah milik putus asa
2          1  jujur aku takut mati aku orang aku sulit paham...
3          0              saya mes pasang sepatu jam saya harap
4          1  capek hati banget aku allah aku kayak mata ora...
...      ...                                                ...
50518      0  curi rampok liar begal rakyat senantiasa resah...
50519      0                      saya bicara saya buat tertawa
50520      1  sakit sakit guna sakit bahagia sakit bohong or...
50521      1                              lantai lantai gelisah
50522      1  aku tulis catat kemarin kata tunang aku laku b...

[50523 rows x 2 columns]

In [9]:
x = df['detokenize'].copy()
y = df['Label'].copy()

## Feature Extraction (TF-IDF)

In [10]:
x

0        aku luka diri aku jalan hidup sulit orang tua ...
1         buang pikir negatif laku gelisah milik putus asa
2        jujur aku takut mati aku orang aku sulit paham...
3                    saya mes pasang sepatu jam saya harap
4        capek hati banget aku allah aku kayak mata ora...
                               ...                        
50518    curi rampok liar begal rakyat senantiasa resah...
50519                        saya bicara saya buat tertawa
50520    sakit sakit guna sakit bahagia sakit bohong or...
50521                                lantai lantai gelisah
50522    aku tulis catat kemarin kata tunang aku laku b...
Name: detokenize, Length: 50523, dtype: object

In [11]:
%%time
# define tf-idf
tf_idf_vec = TfidfVectorizer(use_idf=True, smooth_idf=False,
                             ngram_range=(1,1), max_features=5000)

#create dataframe
x_uni = pd.DataFrame(tf_idf_vec.fit_transform(x).toarray(), columns=tf_idf_vec.get_feature_names_out())
y_uni = y

x_uni

CPU times: user 1.13 s, sys: 1.34 s, total: 2.46 s
Wall time: 2.46 s


aba  abad  abadi  abah  abai  abang  abar  abdi  abis  aborsi  ...  \
0      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
1      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
2      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
3      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
4      0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
...    ...   ...    ...   ...   ...    ...   ...   ...   ...     ...  ...   
50518  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
50519  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
50520  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
50521  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   
50522  0.0   0.0    0.0   0.0   0.0    0.0   0.0   0.0   0.0     0.0  ...   

       zaman       zat  zen  zero  zikir  zina  zodiak  zona  zonasi  zone  
0        0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
1        0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
2        0.0  0.188183  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
3        0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
4        0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
...      ...       ...  ...   ...    ...   ...     ...   ...     ...   ...  
50518    0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
50519    0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
50520    0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
50521    0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  
50522    0.0  0.000000  0.0   0.0    0.0   0.0     0.0   0.0     0.0   0.0  

[50523 rows x 5000 columns]

## Feature Expansion (FastText)

In [12]:
import gensim
from gensim.models import FastText

In [13]:

from gensim.models import FastText

# Sesuaikan path ke lokasi file Anda di Google Drive
file_path = 'corpus_indonews_twitter.fasttext'

# Load model FastText
corpus_indonews = FastText.load(file_path)

In [14]:
similar_word_news = corpus_indonews.wv.most_similar("sedih", topn = 10)
similar_word_news

[('kecewa', 0.7592635750770569),
 ('pedih', 0.7580711841583252),
 ('sesal', 0.6346572637557983),
 ('tangis', 0.6192872524261475),
 ('rasa', 0.6064337491989136),
 ('marah', 0.5997285842895508),
 ('bingung', 0.5642061233520508),
 ('kesal', 0.5555573105812073),
 ('senang', 0.5447505116462708),
 ('sumarah', 0.541563093662262)]

In [15]:
def feature_expansion(df, feature):
    for col in df.columns: #loop per kolom
        try:
            sim_word = corpus_indonews.wv.most_similar(col, topn = 1) #mencari Similarity
        except:
            sim_word = []
        if sim_word != []: #kalo similarity-nya tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    condition = (df[col] == 0) & (df[term] != 0)
                    df.loc[condition, col] = df.loc[condition, term]
    return df

In [16]:
feature_ef = tf_idf_vec.get_feature_names_out()
x_uni_fe = feature_expansion(x_uni, feature_ef)

## Modelling

In [17]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [18]:
from tensorflow.keras.layers import LSTM, Dense, GlobalMaxPool1D, LeakyReLU, Input, Dropout, SpatialDropout1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import tensorflow as tf

def CNN_LSTM(x, y, testsize):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=testsize, random_state=0)

    # Reshape data to (samples, timesteps, features)
    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)

    # Define CNN-LSTM model
    cnn_lstm_model = Sequential()
    cnn_lstm_model.add(Input((X_train.shape[1], X_train.shape[2])))  # Input shape
    cnn_lstm_model.add(SpatialDropout1D(0.2))  # Dropout for regularization
    cnn_lstm_model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))  # Convolution layer
    cnn_lstm_model.add(MaxPooling1D(pool_size=2))  # Max pooling
    cnn_lstm_model.add(Dropout(0.5))  # Dropout
    cnn_lstm_model.add(LSTM(128, return_sequences=True))  # LSTM layer
    cnn_lstm_model.add(Dropout(0.5))  # Dropout
    cnn_lstm_model.add(Flatten())  # Flatten layer
    cnn_lstm_model.add(Dense(32, activation='relu'))  # Dense layer
    cnn_lstm_model.add(Dense(1, activation='sigmoid'))  # Output layer

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    cnn_lstm_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Print model summary
    print(cnn_lstm_model.summary())

    # Train the model
    history = cnn_lstm_model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=64,
        validation_data=(X_test, y_test),
        callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
    )

    # Evaluate the model
    score = cnn_lstm_model.evaluate(X_test, y_test, batch_size=64, verbose=0)
    print()
    print('Validation Accuracy:', score[1])
    print('Validation Loss:', score[0])
    print()

    # Predict and evaluate
    y_pred = cnn_lstm_model.predict(X_test)
    y_true = y_test

    print("Classification Report :")
    classreport = classification_report(y_true, y_pred.round(), digits=4)
    accscore = accuracy_score(y_true, y_pred.round())
    precscore = precision_score(y_true, y_pred.round())
    recscore = recall_score(y_true, y_pred.round())
    f1score = f1_score(y_true, y_pred.round(), average='weighted')

    print('Accuracy Score  : ', accscore)
    print('Precision Score : ', precscore)
    print('Recall Score    :', recscore)
    print('F1 Score        :', f1score)
    print('Classification Report :')
    print(classreport)
    print()

    return [accscore, precscore, recscore, f1score]

## Split Data (80:20)

In [19]:
%%time
hasil = []
hasil.append(['1'] + CNN_LSTM(x_uni_fe, y_uni, 0.20))
hasil.append(['2'] + CNN_LSTM(x_uni_fe, y_uni, 0.20))
hasil.append(['3'] + CNN_LSTM(x_uni_fe, y_uni, 0.20))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spatial_dropout1d               │ (None, 5000, 1)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4996, 128)      │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2498, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2498, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 2498, 128)      │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2498, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 319744)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │    10,231,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,364,225 (39.54 MB)

 Trainable params: 10,364,225 (39.54 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spatial_dropout1d_2             │ (None, 5000, 1)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 4996, 128)      │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 2498, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 2498, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 2498, 128)      │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 2498, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 319744)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │    10,231,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,364,225 (39.54 MB)

 Trainable params: 10,364,225 (39.54 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 100s 156ms/step - accuracy: 0.6466 - loss: 0.5812 - val_accuracy: 0.8113 - val_loss: 0.3859
Epoch 2/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 98s 155ms/step - accuracy: 0.7596 - loss: 0.4345 - val_accuracy: 0.8213 - val_loss: 0.3635
Epoch 3/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 98s 155ms/step - accuracy: 0.7694 - loss: 0.4075 - val_accuracy: 0.8182 - val_loss: 0.3661
Epoch 4/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 98s 155ms/step - accuracy: 0.7856 - loss: 0.3821 - val_accuracy: 0.8211 - val_loss: 0.3664
Epoch 5/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 98s 155ms/step - accuracy: 0.7984 - loss: 0.3583 - val_accuracy: 0.8230 - val_loss: 0.3681

Validation Accuracy: 0.8229589462280273
Validation Loss: 0.36808961629867554

316/316 ━━━━━━━━━━━━━━━━━━━━ 19s 58ms/step
Classification Report :
Accuracy Score  :  0.8229589312221672
Precision Score :  0.837731144816123
Recall Score    : 0.8
F1 Score        : 0.8228596886614551
Classification Report :
              precision    recall  

In [22]:
import os

# Membuat folder 'hasil' jika belum ada
if not os.path.exists('hasil'):
    os.makedirs('hasil')

In [23]:
df = pd.DataFrame(hasil, columns =['Run', 'Accuracy', 'Precision','Recall','F1-Score'])
df.to_excel('hasil/CNN_TF-IDF UniBigram_9010_FastText_Corpus Indonews_Top1.xlsx', index=False)
print('Average Accuracy  :', Average(df['Accuracy'].to_list()))
print('Average Precision :', Average(df['Precision'].to_list()))
print('Average Recall    :', Average(df['Recall'].to_list()))
print('Average F1-Score  :', Average(df['F1-Score'].to_list()))

Average Accuracy  : 0.820451921491011
Average Precision : 0.8244804513461972
Average Recall    : 0.8134920634920636
Average F1-Score  : 0.8204141824943966
